In [1]:
import pandas as pd

In [2]:
# STRING distance (title, album and artist...)
import Levenshtein as lev

def string_distance(string1: str, string2: str):
  # x = remove_non_alphanum(string1).lower()
  # y = remove_non_alphanum(string2).lower()
  x = string1
  y = string2
  dist = lev.distance(x, y) 

  max_len = max(len(x), len(y))
  if (max_len == 0):
    return 0
  normalized = (max_len-dist) / max_len 
  normalized_dist = 1-normalized
  return normalized_dist

In [3]:
title_w = 1.2
album_w = 1
artist_w = 1
number_track_w = .8
year_w = .5

def distance(item: pd.Series, item2: pd.Series): 
  w_sum = 0

  title_dist = 0
  album_dist = 0
  artist_dist = 0
  
  if (item['title'] and item2['title']):
    title_dist = string_distance(item['title'], item2['title'])
    w_sum += title_w

  if (item['album'] and item2['album']):
    album_dist = string_distance(item['album'], item2['album'])
    w_sum += album_w

  if (item['artist'] and item2['artist']):
    artist_dist = string_distance(item['artist'], item2['artist'])
    w_sum += artist_w

  dist = ( (title_dist*title_w) + (album_dist*album_w) + (artist_dist*artist_w) ) / w_sum
  return dist

In [4]:
data = {
    'TID': ['1', '2', '3', '4', '5', '6', '7', '8'],
    'title': ['Boehmian Rhapsody', 'Hotel California', 'Imagine', 'Shape of You', 'Rolling in the Deep', 'Imagine', 'Bohemian Rhapsody', 'Shape of You'],
    'artist': ['Queen', 'Eagles', 'John Lennon', 'Ed Sheran', 'Adele', 'John Lennon', 'Queen', 'Ed Sheeran'],
    'album': ['A Night at the Opera', 'Hotel California', 'Imagine', '÷', '21', 'Imagine Deluxe', 'Greatest Hits', '÷']
}
df = pd.DataFrame(data)
print(df)

  TID                title       artist                 album
0   1    Boehmian Rhapsody        Queen  A Night at the Opera
1   2     Hotel California       Eagles      Hotel California
2   3              Imagine  John Lennon               Imagine
3   4         Shape of You    Ed Sheran                     ÷
4   5  Rolling in the Deep        Adele                    21
5   6              Imagine  John Lennon        Imagine Deluxe
6   7    Bohemian Rhapsody        Queen         Greatest Hits
7   8         Shape of You   Ed Sheeran                     ÷


In [5]:

from modules.PhonexStaticBlocking import PhonexStaticBlocking

blocker = PhonexStaticBlocking(df, 'title', 10)
blocks = blocker.get_blocks()


In [6]:
from modules.CustomKmeans import CustomKmeans 

for i, block in enumerate(blocks):
  print('BLOCK ', i)
  customKmeans = CustomKmeans(df=block, distanceFn=distance, uID='TID', threshold=0.35)

  clusters, clusters_dists = customKmeans.run()

  for cluster_key in clusters.keys():
    print('CLUSTER: ' , cluster_key)
    for i, item in enumerate(clusters[cluster_key]):
      print(item['TID'], item['title'], item['artist'],  item['album'], )
    print('--- dist -->', clusters_dists[cluster_key])
    print()
  print('-----------------------------')


BLOCK  0
CLUSTER:  4
4 Shape of You Ed Sheran ÷
8 Shape of You Ed Sheeran ÷
--- dist --> [0.0, 0.031249999999999993]

CLUSTER:  1
1 Boehmian Rhapsody Queen A Night at the Opera
7 Bohemian Rhapsody Queen Greatest Hits
--- dist --> [0, 0.30974264705882354]

CLUSTER:  2
2 Hotel California Eagles Hotel California
--- dist --> [0]

CLUSTER:  3
3 Imagine John Lennon Imagine
6 Imagine John Lennon Imagine Deluxe
--- dist --> [0, 0.15625]

CLUSTER:  5
5 Rolling in the Deep Adele 21
--- dist --> [0]

-----------------------------
